In [1]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [2]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.0271, 0.1252, 0.9499],
        [0.4255, 0.9384, 0.0949],
        [0.1966, 0.7575, 0.7659],
        [0.1262, 0.3051, 0.1914],
        [0.4049, 0.3843, 0.9447]])


In [4]:
torch.cuda.is_available()

True

In [5]:
#! pip install transformers
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Sun Sep 18 00:16:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   31C    P0    33W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-multi")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-2B-multi")

Downloading:   0%|          | 0.00/240 [00:00<?, ?B/s]

ValueError: Tokenizer class CodeGenTokenizer does not exist or is not currently imported.